## Q. 기상 실측 데이터를 토대로 태양광 발전량인 amount(2023-10-15) 값을 예측해보시오.

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go
from torch.utils.data import TensorDataset, DataLoader

pd.set_option('display.max_column', 20)
pd.set_option('display.max_row', 100)

data = pd.read_csv('./energy_tobigs_question.csv')
data['time'] = pd.to_datetime(data['time'])

# 피처와 타겟 변수 설정
features = ['cloud', 'temp', 'humidity', 'ground_press', 'wind_speed', 'wind_dir', 'rain', 'snow', 'dew_point', 'vis', 'uv_idx', 'azimuth', 'elevation']
target = 'amount'

# 데이터셋 분리 (2023-10-14까지 학습, 2023-10-15 하루 예측)
train_data = data[data['time'] < '2023-10-15'].copy()
test_data = data[(data['time'] >= '2023-10-15') & (data['time'] < '2023-10-16')].copy()

# 학습 데이터에 NaN 값 제거
train_data = train_data.dropna()

# 데이터 스케일링
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()

train_data.loc[:, features] = scaler_x.fit_transform(train_data[features])
train_data.loc[:, target] = scaler_y.fit_transform(train_data[[target]])

test_data.loc[:, features] = scaler_x.transform(test_data[features])

### 시퀀스 데이터 생성 함수 작성

In [ ]:
# 텐서로 변환해서 텐서데이터셋을 생성하세요. (1~6 번에 내용을 채워보세요.)

# 시퀀스 데이터 생성 함수
# dataX: 시퀀스 데이터를 담은 리스트 (예: [[시퀀스1], [시퀀스2], ...])
# dataY: 각 시퀀스에 대응하는 타겟 값 (예: 발전량)

def build_dataset(time_series, seq_length):
    dataX, dataY = [], []
    for i in range(len(time_series) - seq_length):
        dataX.append(time_series[i:i+seq_length, :-1])
        dataY.append(time_series[i+seq_length, [-1]])
    return np.array(dataX), np.array(dataY)

# 하이퍼파라미터 설정
seq_length = ____(1)
batch_size = ____(2)

# 학습 데이터 생성
# trainX: 24시간의 피처 데이터
# trainY: 그 다음 시간에 대한 발전량 값
trainX, trainY = build_dataset(____(3)[features + [target]].values, seq_length)

# 텐서로 변환
trainX_tensor = torch.____(4)(trainX)
trainY_tensor = torch.____(5)(trainY)

# 텐서데이터셋 생성
train_dataset = TensorDataset(trainX_tensor, trainY_tensor)
train_loader = DataLoader(train_dataset, batch_size=____(6), shuffle=True)

### 학습한 시계열 모델을 사용해서 예측을 해보세요 (RNN, LSTM, Transformer, Informer 중에서 선택적으로 활용)

In [ ]:
# 슬라이딩 윈도우 방식으로 2023-10-15의 24시간 발전량 예측해보기 (Direct Multi-step Forecast Strategy 혹은 Recursive Multi-step Forecast 등 이외의 방법론도 자유롭게 사용 가능)

### 2023-10-15 발전량 예측값을 predicted_amounts에 저장하고 시각화를 실행해주세요.

In [ ]:
# 시각화
fig = go.Figure()

# 실제 발전량 시각화
fig.add_trace(go.Scatter(x=data['time'], y=data['amount'],
                         mode='lines', name='Actual Amount'))

# 예측된 발전량 시각화 (2023-10-15의 24시간 동안의 예측)
fig.add_trace(go.Scatter(x=test_data['time'], y=predicted_amounts,
                         mode='lines', name='Predicted Amount', line=dict(dash='dot', color='red')))

# 레이아웃 설정
fig.update_layout(title='2023-10-15 24시간 발전량 예측',
                  xaxis_title='시간',
                  yaxis_title='발전량 (kWh)')

# 그래프 출력
fig.show()

### 정답을 적어주세요.

In [ ]:
# 예측된 24시간 발전량 출력
print(f'2023-10-15 예측 발전량 (24시간): {predicted_amounts} kWh')

---

### (필수) Informer모델은 transformer 모델의 어떤 부분을 개선하고자 했나요? (차이점을 중심으로 서술)

### (필수) 모델링 해석

In [ ]:
# 모델을 선택했다면 왜 선택했는지 본인만의 근거를 정리해주세요.
# 더 나아가 파라미터 선택의 기준이 있었다면 좋습니다.

### (선택) 데이터 해석

In [ ]:
# 데이터셋을 보고 느낀 생각이나 본인만의 논리 전개 방식을 정리해주세요.
# 전처리를 했다면 해당 전처리를 왜 했는지, 파생변수를 생성했다면 왜 만들었는지 등